# Nội dung

[1 Exploring Train](#1-Exploring-Train)

[2 Exploring Questions](#2-Exploring-Questions)

[3 Exploring Lectures](#3-Exploring-Lectures)

[4 Train pipeline](#4-Train-pipeline)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
from matplotlib.ticker import FuncFormatter

import os
for dirname, _, filenames in os.walk('/kaggle/input/riiid-test-answer-prediction'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%%time

train = pd.read_pickle("../input/riid-multi-format-train-set/riiid_train.pkl.gzip")

print("Train size:", train.shape)

Kiểm tra mỗi column chiếm bao nhiêu bộ nhớ

In [ ]:
train.memory_usage(deep=True)

In [ ]:
train.info()

Cột prior_question_had_explanation để dạng object tốn nhiều bộ nhớ > chuyển về bool 

In [ ]:
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('boolean')

train.memory_usage(deep=True)

In [ ]:
train.info()

Tổng bộ nhớ đã giảm đi khoảng 600Mb khi đổi kiểu dữ liệu của prior_question_had_explanation về bool

In [ ]:
%%time

questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
example_test = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')
example_sample_submission = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv')

# 1 Exploring Train

The columns in the train file are described as:
* row_id: (int64) ID của row.
* timestamp: (int64) thời gian tính bằng mili giây giữa lần tương tác của người dùng này và sự kiện đầu tiên hoàn thành từ người dùng đó.
* user_id: (int32) id người dùng.
* content_id: (int16) mã ID cho tương tác của người dùng
* content_type_id: (int8) 0 nếu sự kiện là một câu hỏi được đặt ra cho người dùng, 1 nếu sự kiện là người dùng đang xem một bài giảng.
* task_container_id: (int16) Mã id cho loạt câu hỏi hoặc bài giảng. Ví dụ: một người dùng có thể thấy ba câu hỏi liên tiếp trước khi xem giải thích cho bất kỳ câu hỏi nào trong số đó. Ba câu đó sẽ chia sẻ một task_container_id.
* user_answer: (int8) câu trả lời của người dùng cho câu hỏi, nếu có. Đọc -1 là null cho các bài giảng.
* answer_correctly: (int8) nếu người dùng trả lời đúng. Đọc -1 là null cho các bài giảng.
* prior_question_elapsed_time: (float32) Thời gian trung bình tính bằng mili giây người dùng trả lời từng câu hỏi trong gói câu hỏi trước đó, bỏ qua bất kỳ bài giảng nào ở giữa. Không có giá trị cho gói câu hỏi hoặc bài giảng đầu tiên của người dùng. Lưu ý rằng thời gian là thời gian trung bình mà người dùng dành để giải quyết từng câu hỏi trong nhóm trước đó.
* prior_question_had_explanation: (bool) Người dùng có thấy lời giải thích và (các) câu trả lời chính xác hay không sau khi trả lời gói câu hỏi trước đó, bỏ qua bất kỳ bài giảng nào ở giữa. Giá trị được chia sẻ trên một gói câu hỏi và không có giá trị đối với gói câu hỏi hoặc bài giảng đầu tiên của người dùng. Thông thường, một số câu hỏi đầu tiên mà người dùng nhìn thấy là một phần của kiểm tra chẩn đoán tích hợp trong đó họ không nhận được bất kỳ phản hồi nào.



In [ ]:
train.head(10)

In [ ]:
print(f'Số user: {train.user_id.nunique()}')

Content_type_id = False nếu user trả lời câu hỏi và = True khi user xem lecture.

In [ ]:
train.content_type_id.value_counts()

Content_id là mã đại diện cho tương tác của người dùng, mã này đại diện cho câu hỏi khi mà content_type = False(ám chỉ câu hỏi) và ngược lại là cho lecture

In [ ]:
print(f'Có tất cả {train.content_id.nunique()} content_id unique trong train set trong đó content_id là câu hỏi có {train[train.content_type_id == False].content_id.nunique()} row.')

In [ ]:
cids = train.content_id.value_counts()[:20]

fig = plt.figure(figsize=(12,6))
ax = cids.plot.bar()
plt.title("Top 20 content_id(câu hỏi) xuất hiện nhiều nhất")
plt.xticks(rotation=90)
plt.xlabel("content_id")
plt.ylabel("count")
ax.get_yaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ',')))
plt.show()

task_container_id: (int16) id cho loạt câu hỏi hoặc lecture. Ví dụ: một người dùng có thể thấy ba câu hỏi liên tiếp trước khi xem giải thích cho bất kỳ câu hỏi nào trong số đó. Ba câu hỏi đó sẽ chia sẻ một task_container_id .

In [ ]:
print(f'Có {train.task_container_id.nunique()} loạt câu hỏi hoặc lecture unique')

user_answer. Các câu hỏi là trắc nghiệm (đáp án 0-3). Như phần mô tả dữ liệu ban đầu thì -1 không có câu trả lời (khi tương tác là lecture không phải câu hỏi).

In [ ]:
train.user_answer.value_counts()

timestamp: (int64)thời gian tính bằng mili giây giữa lần tương tác của người dùng đến khi hoàn thành sự kiện đầu tiên từ người dùng đó. Có thể thấy, hầu hết các tương tác là từ những người dùng chưa hoạt động lâu trên hệ thống

In [ ]:
#1 year = 31536000000 ms
ts = train['timestamp']/(31536000000/12)
fig = plt.figure(figsize=(12,6))
ts.plot.hist(bins=100)
plt.title("Histogram of timestamp(by month)")
plt.xticks(rotation=0)
plt.xlabel("Số tháng giữa lần tương tác của người dùng và sự kiện hoàn thành đầu tiên từ người dùng đó ")
plt.show()

In [ ]:
print(f'Có {train[train.timestamp == 0].user_id.nunique()}/{train.user_id.nunique()} user có timestamp bắt đầu từ 0.')

Có thể thấy dữ liệu này thể hiện đầy đủ lịch sử tương tác của user với hệ thống, không có trường hợp chen giữa quá trình

# answered_correctly
answered_correctly là mục tiêu để dự đoán, nếu không tính các trường hợp tương tác là xem lecture thì có khoảng 1/3 số câu trả lời là sai

In [ ]:
correct = train[train.answered_correctly != -1].answered_correctly.value_counts(ascending=True)

fig = plt.figure(figsize=(12,4))
correct.plot.barh()
for i, v in zip(correct.index, correct.values):
    plt.text(v, i, '{:,}'.format(v), color='white', fontsize=14, ha='right', va='center')
plt.title("Questions answered correctly")
plt.xticks(rotation=0)
plt.xlabel("count")
plt.ylabel("label")
plt.show()

In [ ]:
def correct(field):
    correct = train[train.answered_correctly != -1].groupby([field, 'answered_correctly'], as_index=False).size()
    correct = correct.pivot(index= field, columns='answered_correctly', values='size')
    correct['Percent_correct'] = round(correct.iloc[:,1]/(correct.iloc[:,0] + correct.iloc[:,1]),2)
    correct = correct.sort_values(by = "Percent_correct", ascending = False)
    correct = correct.iloc[:,2]
    return(correct)

Những người dùng đã đăng ký tương đối gần đây hoạt động kém hơn một chút so với những người dùng hoạt động lâu hơn. Chia dataset thành 5 bin theo  col timestamp

In [ ]:
bin_labels_5 = ['Bin_1', 'Bin_2', 'Bin_3', 'Bin_4', 'Bin_5']
train['ts_bin'] = pd.qcut(train['timestamp'], q=5, labels=bin_labels_5)

bins_correct = correct("ts_bin")
bins_correct = bins_correct.sort_index()

fig = plt.figure(figsize=(12,6))
plt.bar(bins_correct.index, bins_correct.values)
for i, v in zip(bins_correct.index, bins_correct.values):
    plt.text(i, v, v, color='white', fontsize=14, va='top', ha='center')
plt.title("Percent answered_correctly for 5 bins of timestamp")
plt.xticks(rotation=0)
plt.show()

Phân phối của tỷ lệ trả lời đúng câu hỏi theo task_container_id

In [ ]:
task_id_correct = correct("task_container_id")
fig = plt.figure(figsize=(12,6))
task_id_correct.plot.hist(bins=40)
plt.title("Histogram of percent_correct grouped by task_container_id")
plt.xticks(rotation=0)
plt.show()

In [ ]:
user_percent = train[train.answered_correctly != -1].groupby('user_id')['answered_correctly'].agg(Mean='mean', Answers='count')
sns.boxplot( x=user_percent.Answers);
plt.show()

- Dựa vào boxplot có thể thấy điểm bắt đầu xuất hiện outlier là từ khoảng 1000 nên chọn chặn trên tại điểm này để biểu diễn với sample=1000
- Xét biểu đồ tương quan giữa tỷ lệ trả lời đúng và số lượng câu trả lời của user bên dưới, có thể thấy xu hướng tỷ lệ trả lời đúng tăng lên theo số lượng câu trả lời

In [ ]:
user_percent = user_percent.query('Answers <= 1000').sample(n=1000, random_state=1)

fig = plt.figure(figsize=(12,6))
x = user_percent.Answers
y = user_percent.Mean
plt.scatter(x, y, marker='o')
plt.title("Percent answered correctly vs number of questions answered User")
plt.xticks(rotation=0)
plt.xlabel("Number of questions answered")
plt.ylabel("Percent answered correctly")
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

plt.show()


In [ ]:
content_percent =  train[train.answered_correctly != -1].groupby('content_id')['answered_correctly'].agg(Mean='mean', Answers='count')
sns.boxplot( x=content_percent.Answers);
plt.show()

In [ ]:
print(f'Trên tổng số {len(content_percent)} câu hỏi có {len(content_percent[content_percent.Answers > 25000])} được trả lời nhiều hơn 25,000 lần')

- Dựa vào boxplot có thể thấy điểm bắt đầu xuất hiện outlier là từ khoảng 25000 nên chọn chặn trên tại điểm này để biểu diễn với sample=1000
-  Xét biểu đồ tương quan giữa tỷ lệ trả lời đúng và *số lần được trả lời của 1 content_id(câu hỏi)* bên dưới có thể thấy xu hướng tỷ lệ trả lời đúng giảm dần theo số lượng câu trả lời

In [ ]:
content_percent = content_percent.query('Answers <= 25000').sample(n=1000, random_state=42)

fig = plt.figure(figsize=(12,6))
x = content_percent.Answers
y = content_percent.Mean
plt.scatter(x, y, marker='o')
plt.title("Percent answered correctly versus number of questions answered Content_id")
plt.xticks(rotation=0)
plt.xlabel("Number of questions answered")
plt.ylabel("Percent answered correctly")
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")

plt.show()


- Được xem các gợi ý trước khi trả lời(prior_question_had_explanation) có tác dụng tới kết quả trả lời câu hỏi hay không? Có thể thấy phần trăm trả lời đúng cao hơn khoảng 17% khi có gợi ý.
- Ngoài ra, cũng thấy rằng phần trăm được trả lời đúng cho các giá trị còn Nan gần True hơn False. 

In [ ]:
train.prior_question_had_explanation.value_counts()

In [ ]:
pq = train[train.answered_correctly != -1].groupby(['prior_question_had_explanation'], dropna=False).agg({'answered_correctly': ['mean', 'count']})
# pq.index = pq.index.astype(str)
print(pq)
pq = pq.iloc[:,0]
fig = plt.figure(figsize=(12,4))
pq.plot.barh()
plt.title("Answered_correctly versus Prior Question had explanation")
plt.xlabel("Percent answered correctly")
plt.ylabel("Prior question had explanation")
plt.xticks(rotation=0)
plt.show()

- prior_question_elapsed_time: Có thể thấy thời gian trung bình để đưa ra câu trả lời của user, có thể thấy không có sự khác biết giữa nhãn 0 và 1( đều ở khaongr 25s)

In [ ]:
pq = train[train.answered_correctly != -1]
pq = pq[['prior_question_elapsed_time', 'answered_correctly']]
pq = pq.groupby(['answered_correctly']).agg({'answered_correctly': ['count'], 'prior_question_elapsed_time': ['mean']})
pq

- Nhưng dựa vào biểu đồ dưới đây với sample = 1000 ta thấy được có 1 xu hướng đi xuống của tỷ lệ trả lời đúng > có thể dùng mean của prior_question_elapsed_time là 1 feature

In [ ]:
mean_pq = train.prior_question_elapsed_time.astype("float64").mean()

condition = ((train.answered_correctly != -1) & (train.prior_question_elapsed_time.notna()))
pq = train[condition][['prior_question_elapsed_time', 'answered_correctly']].sample(n=1000, random_state=42)
pq = pq.set_index('prior_question_elapsed_time').iloc[:,0]

fig = plt.figure(figsize=(12,6))
x = pq.index
y = pq.values
plt.scatter(x, y, marker='o')
plt.title("Answered_correctly versus prior_question_elapsed_time")
plt.xticks(rotation=0)
plt.xlabel("Prior_question_elapsed_time")
plt.ylabel("Answered_correctly")
plt.vlines(mean_pq, ymin=-0.1, ymax=1.1)
plt.text(x= 27000, y=0.4, s='mean')
plt.text(x=80000, y=0.6, s='trend')
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")
plt.show()

# 2 Exploring Questions

* question_id: khóa ngoại cho cột train/test content_id, khi content_id  là câu hỏi (0).
* bundle_id: mã mà các câu hỏi được phân phát cùng nhau.
* correct_answer: câu trả lời cho câu hỏi. Có thể so sánh với cột train user_answer để kiểm tra xem user có đúng hay không.
* part: phần liên quan của bài thi TOEIC.
* tags: một hoặc nhiều mã thẻ chi tiết cho câu hỏi. Ý nghĩa của các thẻ sẽ không được cung cấp, nhưng những mã này đủ để nhóm các câu hỏi lại với nhau. 


In [ ]:
questions.head()

In [ ]:
questions.shape

In [ ]:
questions[questions.tags.isna()]

In [ ]:
questions['tags'] = questions['tags'].astype(str)

tags = [x.split() for x in questions[questions.tags != "nan"].tags.values]
tags = [item for elem in tags for item in elem]
tags = set(tags)
tags = list(tags)
print(f'Có {len(tags)} unique tag')

- Số lượng câu trả lời Đúng và Sai của mỗi câu hỏi

In [ ]:
tags_list = [x.split() for x in questions.tags.values]
questions['tags'] = tags_list
questions.head()

correct = train[train.answered_correctly != -1].groupby(["content_id", 'answered_correctly'], as_index=False).size()
correct = correct.pivot(index= "content_id", columns='answered_correctly', values='size')
correct.columns = ['Wrong', 'Right']
correct = correct.fillna(0)
correct[['Wrong', 'Right']] = correct[['Wrong', 'Right']].astype(int)
questions = questions.merge(correct, left_on = "question_id", right_on = "content_id", how = "left")
questions.head()

In [ ]:
%%time

tags_df = pd.DataFrame()
for x in range(len(tags)):
    df = questions[questions.tags.apply(lambda l: tags[x] in l)]
    df1 = df.agg({'Wrong': ['sum'], 'Right': ['sum']})
    df1['Total_questions'] = df1.Wrong + df1.Right
    df1['Question_ids_with_tag'] = len(df)
    df1['tag'] = tags[x]
    df1 = df1.set_index('tag')
    tags_df = tags_df.append(df1)

tags_df[['Wrong', 'Right', 'Total_questions']] = tags_df[['Wrong', 'Right', 'Total_questions']].astype(int)
tags_df['Percent_correct'] = tags_df.Right/tags_df.Total_questions
tags_df = tags_df.sort_values(by = "Percent_correct")

tags_df.head()

- Lấy ngưỡng tỷ lệ trả lời đúng là 0.6 có thể thấy sự phân biệt rõ ràng về tỷ lệ trả lời đúng giữ 10 câu khó và 10 câu dễ nhất trong tập câu hỏi\

In [ ]:
select_rows = list(range(0,10)) + list(range(178, len(tags_df)))
tags_select = tags_df.iloc[select_rows,4]

fig = plt.figure(figsize=(12,6))
x = tags_select.index
y = tags_select.values
clrs = ['red' if y < 0.6 else 'green' for y in tags_select.values]
tags_select.plot.bar(x, y, color=clrs)
plt.title("Ten hardest and ten easiest tags")
plt.xlabel("Tag")
plt.ylabel("Percent answers correct of questions with the tag")
plt.xticks(rotation=90)
plt.show()

- Nhưng số lượng câu khó có tỷ lệ trả lời đúng thấp thì chỉ có khoảng 250k câu trả lời, ít hơn nhiều các câu khác.

In [ ]:
tags_select = tags_df.sort_values(by = "Total_questions", ascending = False).iloc[:30,:]
tags_select = tags_select["Total_questions"]

fig = plt.figure(figsize=(12,6))
ax = tags_select.plot.bar()
plt.title("Thirty tags with most questions answered")
plt.xticks(rotation=90)
plt.ticklabel_format(style='plain', axis='y')
ax.get_yaxis().set_major_formatter(FuncFormatter(lambda x, p: format(int(x), ','))) 
plt.show()

- Column "part", theo tìm hiểu và dựa vào mô tả dữ liệu thì phần part này liên quan đến bài test TOEIC. Có 200 câu hỏi phải trả lời trong hai giờ ở phần Nghe (khoảng 45 phút, 100 câu hỏi) và Đọc (75 phút, 100 câu hỏi).

 * Phần nghe bao gồm Phần 1-4 (Phần Nghe (khoảng 45 phút, 100 câu hỏi)).

 * Phần đọc bao gồm Phần 5-7 (Phần Đọc (75 phút, 100 câu hỏi)). 

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
ax1 = questions.groupby("part").count()['question_id'].plot.bar()
plt.title("Counts of part")
plt.xlabel("Part")
plt.xticks(rotation=0)

part = questions.groupby('part').agg({'Wrong': ['sum'], 'Right': ['sum']})
part['Percent_correct'] = part.Right/(part.Right + part.Wrong)
part = part.iloc[:,2]

ax2 = fig.add_subplot(212)
plt.bar(part.index, part.values)
for i, v in zip(part.index, part.values):
    plt.text(i, v, round(v,2), color='white', fontweight='bold', fontsize=14, va='top', ha='center')

plt.title("Percent_correct by part")
plt.xlabel("Part")
plt.xticks(rotation=0)
plt.tight_layout(pad=2)
plt.show()

- Dựa vào biểu đồ trên có thể thấy là part 5 có số lượng câu hỏi nhiều nhất và cũng là phần khó nhất

# 3 Exploring Lectures
* Lect_id: khóa ngoại cho cột train/test content_id, khi content_id là lecture(1).
* part: category code for the lecture.
* tag: mã thẻ cho bài giảng. Ý nghĩa của các thẻ sẽ không được cung cấp, nhưng những mã tag này có thể dùng để nhóm các lecture lại với nhau.
* type_of: mô tả ngắn gọn về mục đích của lecture


In [ ]:
lectures.head()

In [ ]:
print(f'Có  {lectures.shape[0]} lecture_id.')

In [ ]:
lect_type_of = lectures.type_of.value_counts()

fig = plt.figure(figsize=(12,6))
plt.bar(lect_type_of.index, lect_type_of.values)
for i, v in zip(lect_type_of.index, lect_type_of.values):
    plt.text(i, v, v, color='black', fontweight='bold', fontsize=14, va='bottom', ha='center')
plt.title("Types of lectures")
plt.xlabel("type_of")
plt.ylabel("Count lecture_id")
plt.xticks(rotation=0)
plt.show()

- Các user xem qua lecture có tỷ lệ trả lời đúng cao hơn là không xem, có thể thấy điều này qua biểu đồ dưới đây

In [ ]:
user_lect = train.groupby(["user_id", "answered_correctly"]).size().unstack()
user_lect.columns = ['Lecture', 'Wrong', 'Right']
user_lect['Lecture'] = user_lect['Lecture'].fillna(0)
user_lect = user_lect.astype('Int64')
user_lect['Watches_lecture'] = np.where(user_lect.Lecture > 0, True, False)

watches_l = user_lect.groupby("Watches_lecture").agg({'Wrong': ['sum'], 'Right': ['sum']})
print(user_lect.Watches_lecture.value_counts())

watches_l['Percent_correct'] = watches_l.Right/(watches_l.Right + watches_l.Wrong)

watches_l = watches_l.iloc[:,2]

fig = plt.figure(figsize=(12,4))
watches_l.plot.barh()
for i, v in zip(watches_l.index, watches_l.values):
    plt.text(v, i, round(v,2), color='white', fontweight='bold', fontsize=14, ha='right', va='center')

plt.title("User watches lectures: Percent_correct")
plt.xlabel("Percent correct")
plt.ylabel("User watched at least one lecture")
plt.xticks(rotation=0)
plt.show()

In [ ]:
batch_lect = train.groupby(["task_container_id", "answered_correctly"]).size().unstack()
batch_lect.columns = ['Lecture', 'Wrong', 'Right']
batch_lect['Lecture'] = batch_lect['Lecture'].fillna(0)
batch_lect = batch_lect.astype('Int64')
batch_lect['Percent_correct'] = batch_lect.Right/(batch_lect.Wrong + batch_lect.Right)
batch_lect['Percent_lecture'] = batch_lect.Lecture/(batch_lect.Lecture + batch_lect.Wrong + batch_lect.Right)
batch_lect = batch_lect.sort_values(by = "Percent_lecture", ascending = False)

print(f'Số lượng bài giảng cao nhất được xem trong một task_container_id: {batch_lect.Lecture.max()}.')

In [ ]:
batch_lect.head()

- Dựa vào biểu đồ tương quan ở dưới, có thể thấy không có sự tương quan hay quan hệ nào giữ tỷ lệ trả lời đúng với tỷ lệ lecture trong các task_container 

In [ ]:
batch = batch_lect.iloc[:, 3:]

fig = plt.figure(figsize=(12,6))
x = batch.Percent_lecture
y = batch.Percent_correct
plt.scatter(x, y, marker='o')
plt.title("Percent lectures in a task_container versus percent answered correctly")
plt.xticks(rotation=0)
plt.xlabel("Percent lectures")
plt.ylabel("Percent answered correctly")

plt.show()


In [ ]:
batch_lect['Has_lecture'] = np.where(batch_lect.Lecture == 0, False, True)
print(f'{batch_lect[batch_lect.Has_lecture == True].shape[0]} task_container_ids có lecture và {batch_lect[batch_lect.Has_lecture == False].shape[0]} task_container_ids không có lecture.')

In [ ]:
batch_lect = batch_lect[['Wrong', 'Right', 'Has_lecture']]
batch_lect = batch_lect.groupby("Has_lecture").sum()
batch_lect['Percent_correct'] = batch_lect.Right/(batch_lect.Wrong + batch_lect.Right)
batch_lect = batch_lect[['Percent_correct']]
batch_lect

- Khi có lecture trong 1 task_container_id không ảnh hưởng tới kết quả trả lời, các task_container_id không có lecture có câu trả lời đúng nhiều hơn khoảng 8% so với các task_container_id  có bài giảng. 

# 4 Train pipeline

In [ ]:
%reset -f

In [ ]:
import numpy as np
import pandas as pd
import riiideducation
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.style as style
style.use('fivethirtyeight')
import seaborn as sns
import os
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import gc
import sys
pd.set_option('display.max_rows', None)

In [ ]:
%%time
cols_to_load = ['row_id', 'user_id', 'answered_correctly', 'content_id', 'prior_question_had_explanation', 'prior_question_elapsed_time']
train = pd.read_pickle("../input/riid-multi-format-train-set/riiid_train.pkl.gzip")[cols_to_load]
train['prior_question_had_explanation'] = train['prior_question_had_explanation'].astype('boolean')

print("Train size:", train.shape)

In [ ]:
%%time

questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')
example_test = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')
example_sample_submission = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv')

In [ ]:
%%time
#adding user features
user_df = train[train.answered_correctly != -1].groupby('user_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
user_df.columns = ['user_id', 'user_questions', 'user_mean']

user_lect = train.groupby(["user_id", "answered_correctly"]).size().unstack()
user_lect.columns = ['Lecture', 'Wrong', 'Right']
user_lect = user_lect[['Lecture']].fillna(0).astype('int8')
#user_lect = user_lect.astype('int8')
user_lect['watches_lecture'] = np.where(user_lect.Lecture > 0, 1, 0)
user_lect = user_lect.reset_index()
user_lect = user_lect[['user_id', 'watches_lecture']]

user_df = user_df.merge(user_lect, on = "user_id", how = "left")
del user_lect
user_df.head()

In [ ]:
%%time
#adding content features
content_df = train[train.answered_correctly != -1].groupby('content_id').agg({'answered_correctly': ['count', 'mean']}).reset_index()
content_df.columns = ['content_id', 'content_questions', 'content_mean']
content_df.head()

- Split train/val set sử dụng data của tác giả tito(https://www.kaggle.com/its7171/cv-strategy)

In [ ]:
%%time
cv2_train = pd.read_pickle("../input/riiid-cross-validation-files/cv2_train.pickle")['row_id']
cv2_valid = pd.read_pickle("../input/riiid-cross-validation-files/cv2_valid.pickle")['row_id']

- convert type của prior_question_elapsed_time sang float64 rồi mới lấy mean để có kết quả chính xác, issue được đề cập tại https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/195032

In [ ]:
train = train[train.answered_correctly != -1]

mean_prior = train.prior_question_elapsed_time.astype("float64").mean()

validation = train[train.row_id.isin(cv2_valid)]
train = train[train.row_id.isin(cv2_train)]

validation = validation.drop(columns = "row_id")
train = train.drop(columns = "row_id")

del cv2_train, cv2_valid
gc.collect()

In [ ]:
label_enc = LabelEncoder()

train = train.merge(user_df, on = "user_id", how = "left")
train = train.merge(content_df, on = "content_id", how = "left")
train['content_questions'].fillna(0, inplace = True)
train['content_mean'].fillna(0.5, inplace = True)
train['watches_lecture'].fillna(0, inplace = True)
train['user_questions'].fillna(0, inplace = True)
train['user_mean'].fillna(0.5, inplace = True)
train['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
train['prior_question_had_explanation'].fillna(False, inplace = True)
label_enc.fit(train['prior_question_had_explanation'])
train['prior_question_had_explanation'] = label_enc.transform(train['prior_question_had_explanation'])
train[['content_questions', 'user_questions']] = train[['content_questions', 'user_questions']].astype(int)
train.sample(5)

In [ ]:
validation = validation.merge(user_df, on = "user_id", how = "left")
validation = validation.merge(content_df, on = "content_id", how = "left")
validation['content_questions'].fillna(0, inplace = True)
validation['content_mean'].fillna(0.5, inplace = True)
validation['watches_lecture'].fillna(0, inplace = True)
validation['user_questions'].fillna(0, inplace = True)
validation['user_mean'].fillna(0.5, inplace = True)
validation['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
validation['prior_question_had_explanation'].fillna(False, inplace = True)
validation['prior_question_had_explanation'] = label_enc.transform(validation['prior_question_had_explanation'])
validation[['content_questions', 'user_questions']] = validation[['content_questions', 'user_questions']].astype(int)
validation.sample(5)

In [ ]:
# features = ['user_questions', 'user_mean', 'content_questions', 'content_mean', 'watches_lecture',
#             'prior_question_elapsed_time', 'prior_question_had_explanation']

features = ['user_questions', 'user_mean', 'content_questions', 'content_mean', 'prior_question_elapsed_time']

train = train.sample(n=10000000, random_state = 1)

y_train = train['answered_correctly']
train = train[features]

y_val = validation['answered_correctly']
validation = validation[features]
params = {'objective': 'binary',
          'metric': 'auc',
          'seed': 2020,
          'learning_rate': 0.1,
          "boosting_type": "gbdt" 
         }
lgb_train = lgb.Dataset(train, y_train, categorical_feature = None)
lgb_eval = lgb.Dataset(validation, y_val, categorical_feature = None)
del train, y_train, validation, y_val
gc.collect()

In [ ]:
%%time
model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=50,
    num_boost_round=10000,
    early_stopping_rounds=8
)

In [ ]:
lgb.plot_importance(model)
plt.show()

In [ ]:
env = riiideducation.make_env()

In [ ]:
iter_test = env.iter_test()


In [ ]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.merge(user_df, on = "user_id", how = "left")
    test_df = test_df.merge(content_df, on = "content_id", how = "left")
    test_df['content_questions'].fillna(0, inplace = True)
    test_df['content_mean'].fillna(0.5, inplace = True)
    test_df['watches_lecture'].fillna(0, inplace = True)
    test_df['user_questions'].fillna(0, inplace = True)
    test_df['user_mean'].fillna(0.5, inplace = True)
    test_df['prior_question_elapsed_time'].fillna(mean_prior, inplace = True)
    test_df['prior_question_had_explanation'].fillna(False, inplace = True)
    test_df['prior_question_had_explanation'] = label_enc.transform(test_df['prior_question_had_explanation'])
    test_df[['content_questions', 'user_questions']] = test_df[['content_questions', 'user_questions']].astype(int)
    test_df['answered_correctly'] =  model.predict(test_df[features])
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])